# Hw1_成都市租房价格分析

## 0 准备

In [4]:
import sys
print(sys.executable)
# Pycharm 重启解决问题

/opt/anaconda3/envs/myenv/bin/python


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

## 1 读取数据

In [9]:
file_data=pd.read_csv('./chengdu_rent.csv')
file_data.head()

,标题,价格,面积,区域,链接
0,整租·卓锦万黛 3室1厅 南,1800 元/月,92.68㎡,龙泉驿-大面-卓锦万黛,/zufang/CD1955917199709306880.html
1,整租·天府半岛七期 3室2厅 西南,1900 元/月,91.28㎡,天府新区-锦江生态带-天府半岛七期,/zufang/CD1955464435363479552.html
2,整租·龙樾熙城 3室2厅 东南,3050 元/月,93.00㎡,金牛-国宾-龙樾熙城,/zufang/CD2404832446492721152.html
3,独栋·龙湖冠寓 成都滨江天街店 【国庆钜惠】理工大一个月起租学生双重优惠 0中介 可月付9 开间,1848-2032 元/月,35.00㎡,仅剩2间,/apartment/49105.html
4,整租·锦城华府 1室1厅 东北,2300 元/月,47.32㎡,青羊-八宝街-锦城华府,/zufang/CD2366568447007539200.html


In [10]:
file_data.tail()

,标题,价格,面积,区域,链接
55,整租·首创万卷山 3室1厅 东南,1890 元/月,90.48㎡,龙泉驿-大面-首创万卷山,/zufang/CD1950749833320988672.html
56,整租·万科翡翠公园 4室2厅 南,3325 元/月,121.00㎡,天府新区-锦江生态带-万科翡翠公园,/zufang/CD1941902546016665600.html
57,整租·锦绣家园 2室1厅 东,1750 元/月,70.00㎡,天府新区-海洋公园-锦绣家园,/zufang/CD1953286359561011200.html
58,整租·金科天籁城 3室1厅 东南,2600 元/月,101.42㎡,成华-龙潭寺-金科天籁城,/zufang/CD1955481008304291840.html
59,整租·世纪光华 3室2厅 南,1900 元/月,109.07㎡,温江-花都大道-世纪光华,/zufang/CD1955215546488193024.html
